In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import gzip
import pickle

In [ ]:
plt.style.use('.mplstyle')

In [ ]:
BASE_DIR = "LAMA/data/"
METRICS_DIR = os.path.join(BASE_DIR, "metrics/")

In [ ]:
def average_metrics(results):
    """Average the metrics over samples"""
    metrics = {'precision': {}, 'recall': {}}
    for k in (1, 5, 10,  50, 100):
        if type(list(results[0]['precision'].keys())[0]) == str:
            k = str(k)
        metrics['precision'][k] = np.mean([res['precision'][k] for res in results])
        metrics['recall'][k] = np.mean([res['recall'][k] for res in results])
    metrics['mrr'] = np.mean([res['rr'] for res in results])
    metrics['samples'] = results
    return metrics

In [ ]:
def metrics_to_df(metrics):
    data = []
    for method in ('bm25plus', 'random'):
        for eval_type in ('collapse', 'full'):
            for metric in ('precision', 'recall', 'mrr'):
                metric_result = metrics['evals'][method][eval_type][metric]
                try:
                    for k, score in metric_result.items():
                        data.append((method, 'local', 'dot', eval_type, metric, str(k), score))
                except:
                    data.append((method, 'local', 'dot', eval_type, metric, "1", metric_result))
                              
    for method_type in ('local', 'global'):
        for eval_type in ('collapse', 'full'):
            for normalized in ('cosine', 'dot'):
                if method_type == "global" and normalized == "dot":
                    continue
                for method, method_results in metrics['evals'][method_type][normalized][eval_type].items():
                    for metric in ('precision', 'recall', 'mrr'):
                        metric_result = method_results[metric]
                        try:
                            for k, score in metric_result.items():
                                data.append((method, method_type, normalized, eval_type, metric, str(k), score))
                        except:
                            data.append((method, method_type, normalized, eval_type, metric, "1", metric_result))
                            
    df = pd.DataFrame(data, columns=['layers', 'norm_type', 'normalization', 'eval', 'metrics', 'k', 'score'])                   
    
    df['layer_type'] = 'A'
    df.loc[df['layers'].str.contains('gradients'), 'layer_type'] = 'gradients'
    df.loc[(df['layers'].str.contains('gradients')) & (df['layers'].str.contains('activations')), 'layer_type'] = 'gradients_and_activations'
    df.loc[(df['layers'] == 'random') | (df['layers'] == 'bm25plus'), 'layer_type'] = 'baseline'
    
    df = df.replace({'gradients':'G', 
                     'activations': 'A',
                     'block.': '', 
                     'encoder': 'E', 
                     'decoder': 'D', 
                     'shared': 'emb', 
                     'random': 'Target-Picker'}, regex=True)
    
    return df

In [ ]:
def visualize_one_experiment(paths, 
                             suffix="", 
                             folder="plots/",
                             show=True,
                             return_df=False):
  
    dfs = []
    for (i, path) in enumerate(paths):
        with gzip.open(METRICS_DIR + path) as f:
            reranker_metrics = pickle.load(f)
            
        df = metrics_to_df(reranker_metrics)
        df = df[~df['layers'].str.contains('A.E.0,A.D.0,')]
        df['seed'] = i
        dfs.append(df)
      
    df = pd.concat(dfs, ignore_index=True)
    # scores = df.groupby(['normalization', 'eval', 'layers', 'metrics', 'k']).agg({'score': ['mean', 'std']}, as_index=False)
    
    # return df, scores
    
    
    if return_df:
        return df
    for method in ("full", "collapse"):
      
      plot_with_filter(df, 
                       filter=lambda x: x[(x['metrics'] == 'precision') & (x['k'] == '3')  & (x['eval'] == method)],
                       title=f'precision@3 ({method} + {suffix})',
                       folder=folder,
                       show=show)
      
      plot_with_filter(df,
                       filter=lambda x: x[(x['metrics'] == 'recall') & (x['k'] == '3') & (x['eval'] == method)],
                       title=f'recall@3 ({method} + {suffix})',
                       folder=folder,
                       show=show)
      
      plot_with_filter(df,
                       filter=lambda x: x[(x['metrics'] == 'mrr') & (x['k'] == '1') & (x['eval'] == method)],
                       title=f'mrr ({method} + {suffix})',
                       folder=folder,
                       show=show)
  

In [ ]:
sdf  = visualize_one_experiment(paths=[f'reranker/unfiltered/seed_{i}/learned/no_eos_accum/results.entity.pickle'
                                     for i in range(1)
                                     ], 
                              suffix="", 
                              show=False, 
                              return_df=True)

In [ ]:
sdf[(sdf['norm_type'] == 'local') & (sdf['normalization'] == 'cosine')]

In [ ]:
sdf[(sdf['norm_type'] == 'global') & (sdf['normalization'] == 'cosine')]

In [ ]:
sdf['layer_type'] = sdf['layer_type'].replace({'G': 'TracIn', 
                           'A': 'Embed', 
                           'G_and_A': 'TracIn+Embed',
                           'baseline': 'Baselines'
                          })

In [ ]:
sdf['layers'] = sdf['layers'].replace({f'G.E.{i},G.D.{i}': f'G.E.{i+1},G.D.{i+1}' for i in range(12)})\
             .replace({f'G.E.{i}': f'G.E.{i+1}' for i in range(12)})\
             .replace({f'G.emb,G.E.{i},G.D.{i}': f'G.emb,G.E.{i+1},G.D.{i+1}' for i in range(12)})\
             .replace({f'G.emb,G.E.{i}': f'G.emb,G.E.{i+1}' for i in range(12)})\
             .str.replace('G.emb', 'G.0').str.replace('bm25plus','BM25+').str.replace('Target-Picker', 'Random-Target')
             

In [ ]:
sdf.head()

In [ ]:
def plot_with_filter(df, filter=lambda x: x, title="Title", folder="plots/", show=True):
    plt.figure(figsize=(16, 6))
    ax = sns.barplot(data=filter(df), 
                x='layers', 
                y='score', 
                hue='layer_type', 
                estimator=np.mean, 
                ci='sd')
    plt.xticks(rotation=90)
    plt.title(title)
    plt.xlabel("Layer Selection")
    plt.ylabel("Recall@10")
    plt.legend(title='Method Type')
    if show:
        plt.show()
    else:
        plt.savefig(folder+title+"_plot.png")

In [ ]:
plot_with_filter(sdf,
                filter=lambda x: x[(x['metrics'] == 'mrr') & (x['k'] == '1') & (x['eval'] == 'collapse') & (x['normalization'] == 'cosine') & (x['norm_type'] == "local")],
                title='',
                show=False,
                folder="./paper_figures/")

In [ ]:
metrics['evals']['global']['cosine']['full']['gradients.']['precision']

In [ ]:
metrics['evals']['local']['cosine']['full']['gradients.']['precision']

In [ ]:
df = {}
for norm_type in ("ln", ):
    for eos in ("eos", ):
        for subset in ("learned", ):
            for accum in ("no_accum", "accum"):
                try:
                    suffix=f"{norm_type}+{eos}+{subset}+{accum}"
                    df[suffix]=visualize_one_experiment(paths=[f'/reranker/exp_layers_{i}/{norm_type}_sl_{eos}__{subset}_{accum}.json'
                                                               for i in range(2)], 
                                                        suffix=suffix, 
                                                        show=False, 
                                                        return_df=True)
                except FileNotFoundError:
                    print(f'notfound: /reranker/exp_layers/{norm_type}_sl_{eos}__{subset}_{accum}.json')

In [ ]:
for (k, v) in df.items():
    norm_type, eos, subset, accum = k.split('+')
    v['norm_type'] = norm_type
    v['eos'] = eos
    v['subset'] = subset
    v['accum'] = accum
    

In [ ]:
dfmerged = pd.concat(df.values(), ignore_index=False)

In [ ]:
dfmerged.head()

In [ ]:
scores = dfmerged.groupby(['normalization', 'eval', 'layers', 'metrics', 'k', 'layer_type', 'norm_type', 'eos', 'subset', 'accum']).agg({'score': ['mean', 'std']}, as_index=False)
scores = scores.reset_index()
    

In [ ]:
scores

In [ ]:
def get_max(df: pd.DataFrame, prefix, no_prefix=None, subset='learned', metric='mrr', eval_type='collapse', k='1'):
    df2 = df[(df['layers'].str.startswith(prefix)) & (df['subset'] == subset) & (df['metrics'] == metric) & (df['k'] == str(k)) & (df['eval'] == eval_type)]
    if no_prefix:
        df2 = df2[~df2['layers'].str.contains(no_prefix)]
    nlargest = df2.nlargest(20, ('score', 'mean'))
    largest = nlargest.iloc[0]    
    dflargest = df[(df['layers'] == largest['layers'][0]) & (df['norm_type'] == largest['norm_type'][0]) & (df['eos'] == largest['eos'][0]) &  (df['accum'] == largest['accum'][0]) &(df['subset'] == subset) & (df['eval'] == eval_type)]
    return nlargest, dflargest


In [ ]:
largest, dflargest = get_max(scores, 'A', no_prefix='G')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'A')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'G')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'bm25')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest  = get_max(scores, 'Target-Picker')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'A', no_prefix='G', subset='random')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'A', subset='random')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'G', subset='random')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'bm25', subset='random')
display(largest)
display(dflargest)

In [ ]:
largest, dflargest = get_max(scores, 'Target-Picker', subset='random')
display(largest)
display(dflargest)

## Eval on correct samples (+eos)

In [ ]:

for norm_type in ("ln", "gn"):
    for eos in ("no_eos", ):
        for subset in ("learned", ):
            for accum in ("accum", ):
                try:
                    # visualize_one_experiment(path=f'/reranker/{norm_type}_sl_{eos}__{subset}_{accum}.json', suffix=f"{norm_type} + {eos} + {subset} + {accum}", show=False, folder="sentence_level_all_plots/")
                    visualize_one_experiment(path=f'/reranker/{norm_type}_sl_{eos}__{subset}_{accum}.json', 
                                             suffix=f"{norm_type} + {eos} + {subset} + {accum}", 
                                             show=True)
                except FileNotFoundError:
                    print(f'notfound: /reranker/{norm_type}_sl_{eos}__{subset}_{accum}.json')

In [ ]:

def get_nn_abstracts(res, config=""):
    """Get list of abstracts and their scores (dotproduct score)"""
    print(f"config: {config}")
    example = res['example']
    print(example['inputs_pretokenized']," -> ", example['targets_pretokenized'])
    fact = (example['predicate_id'].strip(), example['obj_uri'].strip(), example['sub_uri'].strip())
    print(fact)
    abstracts = np.array(res['nn_abstracts'])
    for i, abstract in enumerate(abstracts):
        try:
            abstract['score'] = res['nn_scores'][i]
        except KeyError:
            abstract['score'] = res['nn']['scores'][i]
    df = pd.DataFrame(pd.json_normalize(abstracts)).round(3)
    # df['inputs_pretokenized'] = df['inputs_pretokenized'].str.slice(0,512)
    df['label'] = df['facts'].str.contains(",".join(fact))
    df = df.drop(['page_uri', 'masked_uri', 'masked_type', 'facts', 'sentence_uris'], axis=1)
    return df

def result_getter(path):
    with open(METRICS_DIR + path) as f:
            reranker_metrics = json.load(f)
      
    def getter(i=3,
               sim="cosine",
               method="full",
               layers='activations.encoder.block.11,activations.decoder.block.11'):
        config = {"sim": sim, "method": method, "layers": layers}
        return get_nn_abstracts(reranker_metrics[sim][method][layers]['samples'][i], config=config)
          
    getter.metrics = reranker_metrics
    return getter
            


In [ ]:
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_colwidth', 0)

In [ ]:
getter = result_getter('/reranker/exp_layers_0/ln_sl_no_eos__learned_no_accum.json')

In [ ]:
def side_by_side(df1, df2, df3):
    df = pd.DataFrame(columns=['TracIn','Embed','BM25', ])
    for dfi in (df1, df2, df3):
        dfi['sample'] = "\textbf{Q:} " + dfi['inputs_pretokenized'] + "\n\textbf{A:} " + dfi['targets_pretokenized'].replace('<extra_id_0> ','') + "\n\textbf{" + dfi['label'].astype(str) + "}"
        dfi['sample'] = dfi['sample'].str.replace("<extra_id_0>", "[MASK]")
    df['TracIn'] = df1['sample']
    df['Embed'] = df2['sample']
    df['BM25'] = df3['sample']
    return df
    

In [ ]:
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
pd.__version__

In [ ]:
for i in range(40, 45):
    dfside = side_by_side(getter(layers='gradients.encoder.block.1,gradients.decoder.block.1', i=i),
                 getter(layers='activations.encoder.block.0,activations.decoder.block.0', i=i),
                 getter(layers='bm25plus', i=i)).head(5)
    # Assuming the variable df contains the relevant DataFrame
    display(dfside.style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
    }))
    print(dfside.to_latex(index=False, escape=False))

In [ ]:
i=21
display(getter(layers='gradients.shared', i=i).head(5))
display(getter(layers='activations.encoder.block.0,activations.decoder.block.0', i=i).head(5))
display(getter(layers='bm25plus', i=i).head(5))

In [ ]:
getter2 = result_getter('/reranker/exp_layers_0/ln_sl_no_eos__learned_accum.json')

In [ ]:
display(getter2(layers='gradients.shared'))

In [ ]:
getter = result_getter('/reranker/exp_layers/gn_sl_eos__learned_accum.json')
display(getter(layers='activations.encoder.block.11,activations.decoder.block.11'))
display(getter(layers='activations.encoder.block.0,activations.decoder.block.0'))
display(getter(layers='gradients.encoder.block.11,gradients.decoder.block.11'))
display(getter(layers='gradients.shared'))

In [ ]:
# getter = result_getter('/reranker/exp_layers/gn_sl_eos__corrects_accum.json')
# display(getter(layers='activations.encoder.block.11,activations.decoder.block.11'))
# display(getter(layers='activations.encoder.block.0,activations.decoder.block.0'))
# display(getter(layers='gradients.encoder.block.11,gradients.decoder.block.11'))
# display(getter(layers='gradients.shared'))

In [ ]:
len(getter.metrics['cosine']['full']['bm25plus']['samples'])